In [ ]:
import os
import re
import time
import pandas as pd
import numpy as np
from collections import Counter

import spacy
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import optuna

In [2]:
dataPath = "Data\\track-a.csv"

In [3]:
dataFrame = pd.read_csv(dataPath)

In [4]:
nlpModel = spacy.load("en_core_web_sm", disable=["parser","ner"])

In [5]:
def cleanerFunction(text):
    tempDoc = nlpModel(text)
    token = [
        tok.lemma_.lower()
        for tok in tempDoc
        if not tok.is_stop and not tok.is_punct and tok.lemma_ != "-PRON-"
    ]
    return " ".join(token)

In [6]:
dataFrame["Spacy_text"] = dataFrame["text"].astype(str).apply(cleanerFunction)

In [7]:
labelColumns = [col for col in ["anger", "fear", "joy", "sadness", "surprise"] if col in dataFrame]
y = dataFrame[labelColumns].values

In [9]:
xTrain, xTest, yTrain, yTest = train_test_split(dataFrame["Spacy_text"], y, test_size = 0.1, random_state = 42)

In [14]:
textTrain, textValue, labelTrain, labelValue = train_test_split(dataFrame["Spacy_text"], y, test_size = 0.1, random_state = 42)

In [15]:
textTrain = textTrain.reset_index(drop = True)
textValue = textValue.reset_index(drop = True)

textValue = textValue.tolist()
textTrain = textTrain.tolist()

In [18]:
userDevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")